In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/tp2datostrain/test.csv
/kaggle/input/tp2datostrain/train.csv
/kaggle/input/pingsound/PING - Sound effect.mp3


In [2]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from category_encoders import CountEncoder
from sklearn.model_selection import train_test_split
import xgboost as xgb
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error


In [3]:
from IPython.display import Audio
sound_file = '/kaggle/input/pingsound/PING - Sound effect.mp3'
def ping():
    display(Audio(sound_file, autoplay=True))

In [160]:
import pandas as pd
test = pd.read_csv("../input/tp2datostrain/test.csv")
train = pd.read_csv("../input/tp2datostrain/train.csv")

In [161]:
## Separo un pedazo del test de train para testear luego...
PORCENTAJE_A_SEPARAR = 0.1
filas_totales = train.shape[0]

df_test = train.tail(int(filas_totales * PORCENTAJE_A_SEPARAR)).copy()
df_test.shape

(24000, 23)

In [162]:
dfCheto = pd.concat([train,test], sort=False)
dfCheto['precio'] = SimpleImputer(strategy='median').fit_transform(dfCheto[['precio']])
dfCheto = train.head(int(filas_totales * (100 - PORCENTAJE_A_SEPARAR)))
df = dfCheto.copy()

#train = aux.head(150000)
#test = aux.tail(90000)
#test.drop(columns=['precio'],inplace =True)
#dfCheto = pd.concat([train,test], sort=False)

In [163]:
ciudades = list(dfCheto["ciudad"].unique())
tipos_de_propiedad = list(dfCheto["tipodepropiedad"].unique())
provincia = list(dfCheto["provincia"].unique())
categorias  = [provincia, tipos_de_propiedad, ciudades]

In [164]:
df.columns

Index(['id', 'titulo', 'descripcion', 'tipodepropiedad', 'direccion', 'ciudad',
       'provincia', 'antiguedad', 'habitaciones', 'garages', 'banos',
       'metroscubiertos', 'metrostotales', 'idzona', 'lat', 'lng', 'fecha',
       'gimnasio', 'usosmultiples', 'piscina', 'escuelascercanas',
       'centroscomercialescercanos', 'precio'],
      dtype='object')

In [165]:
aux2 = dfCheto[['ciudad', 'provincia']].copy()
aux2.dropna(inplace = True)
aux2 = aux2.groupby(['provincia']).agg(lambda x:x.provincia.value_counts().index[0]).reset_index().set_index('ciudad')
dicc_ciudadPorPcia = aux2.T.to_dict('records').copy()[0]
dicc_ciudadPorPcia['CUALQUIERA'] = 'CUALQUIERCIUDAD'
df['ciudad'] =df['ciudad'].fillna('CUALQUIERA')
df['provincia'] = df['provincia'].fillna(df['ciudad'].map(dicc_ciudadPorPcia))

In [166]:
df.provincia.fillna('provinciaVacia',inplace=True)

In [167]:
df['provincia'].isna().sum()

0

In [168]:
aux2 = dfCheto[['ciudad', 'provincia']].copy()
aux2 = aux2.groupby(['provincia']).agg(lambda x:x.provincia.value_counts().index[0]).reset_index().set_index('ciudad')
aux2.head()
dicc_ciudadMasFrecuentePorPcia = aux2.T.to_dict('records').copy()[0]
dicc_invertido = dict(map(reversed, dicc_ciudadMasFrecuentePorPcia.items()))
df.ciudad.fillna(df.provincia.map(dicc_invertido),inplace=True);
df.ciudad.isna().sum()

0

In [169]:
df.ciudad.fillna('ciudadVacia',inplace=True)

In [170]:
df.tipodepropiedad.fillna('tipoVacio',inplace=True)
'''
auxmetros = df[['tipodepropiedad','metrostotales']].copy()
auxmetros = auxmetros.groupby('tipodepropiedad')['metrostotales'].mean().sort_values().reset_index()
lista = sorted(auxmetros.metrostotales.tolist())
lista_2 = [0]*len(lista)
for i,numero in enumerate(lista):
    if i == 0: lista_2[i] = (0,numero)
    else:
        lista_2[i] = (lista[i-1],lista[i])
rangoMetros = pd.Series(lista_2).sort_values()
auxmetros['rangoMetros'] = rangoMetros
auxmetros = auxmetros.set_index('tipodepropiedad').drop(columns='metrostotales')
dicc_tipoDeProp = auxmetros.T.to_dict('records').copy()[0]
def mapearMetros(metrostotales,dicc_tipoDeProp):
    for clave in dicc_tipoDeProp:
        if metrostotales >= dicc_tipoDeProp[clave][0] and metrostotales < dicc_tipoDeProp[clave][1]: return clave
    return 'Casa'
df.tipodepropiedad.fillna(df.metrostotales.map(lambda x: mapearMetros(x,dicc_tipoDeProp)),inplace=True);
df.tipodepropiedad.isna().sum()
'''

"\nauxmetros = df[['tipodepropiedad','metrostotales']].copy()\nauxmetros = auxmetros.groupby('tipodepropiedad')['metrostotales'].mean().sort_values().reset_index()\nlista = sorted(auxmetros.metrostotales.tolist())\nlista_2 = [0]*len(lista)\nfor i,numero in enumerate(lista):\n    if i == 0: lista_2[i] = (0,numero)\n    else:\n        lista_2[i] = (lista[i-1],lista[i])\nrangoMetros = pd.Series(lista_2).sort_values()\nauxmetros['rangoMetros'] = rangoMetros\nauxmetros = auxmetros.set_index('tipodepropiedad').drop(columns='metrostotales')\ndicc_tipoDeProp = auxmetros.T.to_dict('records').copy()[0]\ndef mapearMetros(metrostotales,dicc_tipoDeProp):\n    for clave in dicc_tipoDeProp:\n        if metrostotales >= dicc_tipoDeProp[clave][0] and metrostotales < dicc_tipoDeProp[clave][1]: return clave\n    return 'Casa'\ndf.tipodepropiedad.fillna(df.metrostotales.map(lambda x: mapearMetros(x,dicc_tipoDeProp)),inplace=True);\ndf.tipodepropiedad.isna().sum()\n"

In [171]:
aux2 = dfCheto[['idzona', 'ciudad']].copy()
aux2.dropna(inplace = True)
aux2 = aux2.groupby('ciudad').agg({'idzona':'median'})
ids = aux2.T.to_dict('records').copy()
ids = ids[0]
df.idzona.fillna(df.ciudad.map(ids), inplace = True)

In [172]:
df['idzona'] = SimpleImputer(strategy='most_frequent').fit_transform(df[['idzona']])

In [173]:
aux = dfCheto[['lat', 'ciudad']].copy()
aux.dropna(inplace = True)
aux = aux.groupby('ciudad').agg({'lat':'mean'})
lats = aux.T.to_dict('records').copy()
lats = lats[0]
df.lat.fillna(df.ciudad.map(lats), inplace = True)

In [174]:
df['lat'] = SimpleImputer(strategy='most_frequent').fit_transform(df[['lat']])

In [175]:
aux = dfCheto[['lng', 'ciudad']].copy()
aux.dropna(inplace = True)
aux = aux.groupby('ciudad').agg({'lng':'mean'})
lngs = aux.T.to_dict('records').copy()
lngs = lngs[0]
df.lng.fillna(df.ciudad.map(lngs), inplace = True)

In [176]:
df['lng'] = SimpleImputer(strategy='most_frequent').fit_transform(df[['lng']])

In [177]:
aux = dfCheto[['tipodepropiedad', 'habitaciones']].copy()
aux.dropna(inplace = True)
aux = aux.groupby('tipodepropiedad').agg({'habitaciones':'median'})
habts = aux.T.to_dict('records').copy()
habts = habts[0]
df.habitaciones.fillna(df.tipodepropiedad.map(habts), inplace = True)

In [178]:
df['habitaciones'] = SimpleImputer(strategy='most_frequent').fit_transform(df[['habitaciones']])

In [179]:
aux = dfCheto[['tipodepropiedad', 'garages']].copy()
aux.dropna(inplace = True)
aux = aux.groupby('tipodepropiedad').agg({'garages':'median'})
habts = aux.T.to_dict('records').copy()
habts = habts[0]
df.garages.fillna(df.tipodepropiedad.map(habts), inplace = True)

In [180]:
df.garages.fillna(0, inplace=True)

In [181]:
df.drop(columns=['direccion'], inplace=True)

In [182]:
aux = dfCheto[['antiguedad', 'ciudad']].copy()
aux.dropna(inplace = True)
aux = aux.groupby('ciudad').agg({'antiguedad':'mean'})
lngs = aux.T.to_dict('records').copy()
lngs = lngs[0]
df.antiguedad.fillna(df.ciudad.map(lngs), inplace = True)

In [183]:
df['antiguedad'] = SimpleImputer(strategy='most_frequent').fit_transform(df[['antiguedad']])

In [184]:
aux = dfCheto[['banos', 'tipodepropiedad']].copy()
aux.dropna(inplace = True)
aux = aux.groupby('tipodepropiedad').agg({'banos':'median'})
lngs = aux.T.to_dict('records').copy()
lngs = lngs[0]
df.banos.fillna(df.tipodepropiedad.map(lngs), inplace = True)

In [185]:
df.banos.fillna(1, inplace=True)

In [186]:
aux = df.copy()

In [187]:
aux.metrostotales.fillna(0, inplace=True)
aux.metroscubiertos.fillna(0, inplace =True)
aux['metrostotalesCorregidos'] = aux[['metrostotales', 'metroscubiertos']].max(axis=1)
aux['metroscubiertosCorregidos'] = aux[['metrostotales', 'metroscubiertos']].min(axis=1)
df['metrostotales'] = aux['metrostotalesCorregidos']
df['metroscubiertos'] = aux['metroscubiertosCorregidos']

In [188]:
df['fecha'] = pd.to_datetime(df['fecha'])
df['dia'] = df.fecha.dt.day
df['mes'] = df.fecha.dt.month
df['anio'] = df.fecha.dt.year
df.drop(columns=['fecha'], inplace=True)

In [189]:
df['publicacionesPorCiudad'] = CountEncoder().fit_transform(df['ciudad'])

In [190]:
df['publicacionesPorProvincia'] = CountEncoder().fit_transform(df['provincia'])

In [191]:
df['publicacionesPorTipo'] = CountEncoder().fit_transform(df['tipodepropiedad'])
df['publicacionesPorZona'] = CountEncoder().fit_transform(df['idzona'])
df['publicacionesPorBanos'] = CountEncoder().fit_transform(df['banos'])
df['publicacionesPorHabitaciones'] = CountEncoder().fit_transform(df['habitaciones'])
df['publicacionesPorAntiguedad'] = CountEncoder().fit_transform(df['antiguedad'])

In [192]:
df[['tipodepropiedad','provincia','ciudad']].isna().sum()

tipodepropiedad    0
provincia          0
ciudad             0
dtype: int64

### Datos por grupo

In [193]:
df['promedioAntiguedadZona'] = df.groupby('idzona')['antiguedad'].transform(np.mean)
df['promedioAntiguedadCiudad'] = df.groupby('ciudad')['antiguedad'].transform(np.mean)
df['promedioHabitacionesZona'] = df.groupby('idzona')['habitaciones'].transform(np.mean)
df['promedioHabitacionesCiudad'] = df.groupby('ciudad')['habitaciones'].transform(np.mean)
df['promedioMetrosCubiertosZona'] = df.groupby('idzona')['metroscubiertos'].transform(np.mean)
df['promedioMetrosCubiertosCiudad'] = df.groupby('ciudad')['metroscubiertos'].transform(np.mean)
df['promedioMetrosTotalesZona'] = df.groupby('idzona')['metrostotales'].transform(np.mean)
df['promedioMetrosTotalesCiudad'] = df.groupby('ciudad')['metrostotales'].transform(np.mean)
df['promedioPrecioPorProvincia'] = df.groupby('provincia')['precio'].transform(np.mean)
df['promedioPrecioPorIdzona'] = df.groupby('idzona')['precio'].transform(np.mean)
df['promedioPrecioPorCiudad'] = df.groupby('ciudad')['precio'].transform(np.mean)

In [194]:
aux = train[['provincia', 'ciudad', 'idzona','precio', 'metrostotales']].copy()
aux['precioXM2'] = aux['precio'] / aux['metrostotales']
df['promedioPXM2prov'] = aux.groupby('provincia')['precioXM2'].transform(np.mean)
df['promedioPXM2zona'] = aux.groupby('idzona')['precioXM2'].transform(np.mean)
df['promedioPXM2ciudad'] = aux.groupby('ciudad')['precioXM2'].transform(np.mean)
df['ciudadSobreCiudad'] = df['promedioPXM2ciudad'] / df['promedioPXM2prov']
df['promedioPromediosPXM2'] = (0.3*df['promedioPXM2prov'] + 0.4*df['promedioPXM2zona'] + 0.3*df['promedioPXM2ciudad']) / 3

### Texto

In [ ]:
df.titulo.fillna('', inplace=True)
df.descripcion.fillna('', inplace=True)
df["titulo_descripcion"] = df["titulo"].astype(str) + " " + df["descripcion"].astype(str)
#Como tiene algunos nulos los relleno con cadenas vaicas
df["titulo_descripcion"].fillna("",inplace = True)
#REMUEVO LA PUNTUACION
import string
print(string.punctuation)
tabla = {}
for c in string.punctuation + "¿¡":
    tabla[ord(c)] = None
def sacar_puntuacion(s):
    return s.translate(tabla)
df["titulo_descripcion"] = df["titulo_descripcion"].apply(sacar_puntuacion)
#Agrego los primeros features
df["nro_palabras"] = df["titulo_descripcion"].apply(lambda x: len(x.split(" ")))
df["nro_caracteres"] = df["titulo_descripcion"].apply(lambda x: len(x))
df["long_prom_palabra"] = df["nro_caracteres"] / df["nro_palabras"]

def llenarDiccionario(diccionario,descripcion):
    for palabra in descripcion.split():
        diccionario[palabra] = diccionario.get(palabra,0) + 1
        
def generarDiccionario(df2):
    dicc = {}
    for descripcion in df2['titulo_descripcion']:
        if isinstance(descripcion,str): llenarDiccionario(dicc,descripcion)
    return dicc
contador_palabras = generarDiccionario(df)

#Elimino las stopwords = palabras vacias que no suman nada
from nltk.corpus import stopwords
palabras_vacias = stopwords.words('spanish')

for palabra  in palabras_vacias:
    if palabra in contador_palabras:
        del contador_palabras[palabra]
ordenados = sorted(contador_palabras.items(),key = lambda x: -x[1])
top_50 = dict(ordenados[:50])

#Nuevos features
def contar_palabras_en_top(texto):
    palabras = texto.split()
    cont = 0
    for palabra in palabras:
        if palabra in top_50:
            cont += 1
    return cont
df["nro_palabras_mas_comunes"] = df["titulo_descripcion"].apply(contar_palabras_en_top)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [ ]:
df.columns

In [ ]:
features = [    
       'id', 
    'antiguedad',
       'habitaciones', 'garages', 'banos', 'metroscubiertos', 'metrostotales',
       'idzona', 'lat', 'lng', 'gimnasio', 'usosmultiples', 'piscina',
       'escuelascercanas', 'centroscomercialescercanos',
       'dia', 'mes', 'anio',
        #Canti de publicaciones
        'publicacionesPorCiudad',
       'publicacionesPorProvincia', 'publicacionesPorTipo',
        'publicacionesPorZona','publicacionesPorBanos',
        'publicacionesPorHabitaciones', 'publicacionesPorAntiguedad',
            
        #Texto
        "nro_palabras", "nro_caracteres", "long_prom_palabra", "nro_palabras_mas_comunes",
         #Promedios
        'promedioAntiguedadZona', 'promedioAntiguedadCiudad',
        'promedioHabitacionesZona', 'promedioHabitacionesCiudad',
        'promedioMetrosCubiertosZona','promedioMetrosCubiertosCiudad',
        'promedioMetrosTotalesZona', 'promedioMetrosTotalesCiudad', 
        'promedioPrecioPorProvincia', 'promedioPrecioPorIdzona', 
        'promedioPrecioPorCiudad',
        #'promedioPXM2prov',
        'promedioPXM2zona',
        #'promedioPXM2ciudad',
        'ciudadSobreCiudad',
            #'promedioPxM2Provincia',
        
         'ciudad','tipodepropiedad', 'provincia', ]

In [ ]:

transformador_categorias = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore', categories = categorias)),
])

transformador_numeros = Pipeline(steps=[('a', SimpleImputer(strategy = 'median')),
                                        ])

In [ ]:
def obtenerFeaturesCategoricos(features):
    l = []
    for feature in features:
        if not np.issubdtype(df[feature].dtype, np.number):
            l.append(feature)
    return l

num_features = list(df[features].select_dtypes(include=[np.number]).columns)

cat_features = list(obtenerFeaturesCategoricos(features))
print ("Features numericos:", num_features, '\nFeatures categoricos: ', cat_features)

preprocessor = ColumnTransformer(
    transformers=[
        ('num', transformador_numeros, num_features),
        ('cat', transformador_categorias, cat_features)
    ])

In [ ]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', transformador_numeros, num_features),
        ('cat', transformador_categorias, cat_features)
   ])

In [ ]:
model = xgb.XGBRegressor(#learning_rate= 0.1,n_estimators= 1000,scale_pos_weight= 2,max_depth= 5,min_child_weight= 3,gamma= 0.0,colsample_bytree= 0.75,                     subsample= 0.7,colsample_bylevel= 0.65, 
    learning_rate= 0.03,n_estimators= 800,scale_pos_weight= 2,max_depth= 5,min_child_weight= 5, colsample_bytree= 0.5,subsample= 0.5,colsample_bylevel= 0.65, 
    tree_method='gpu_hist', gpu_id=0, eval_metric = "mae")

my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', model)
                             ])

In [ ]:
#entrenamiento = df.head(240000).copy()

In [124]:
#prueba = df.tail(60000).copy()

In [125]:
prueba.drop(columns=['precio'], inplace=True)

In [126]:
entrenamiento = df
train_x = entrenamiento[features].copy()
train_y = entrenamiento.precio.copy()
train_x, test_x, train_y, test_y = train_test_split(train_x, train_y,train_size=0.8, test_size=0.2)
my_pipeline.fit(train_x, train_y)

/opt/conda/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[15:08:28] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


Pipeline(memory=None,
         steps=[('preprocessor',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('num',
                                                  Pipeline(memory=None,
                                                           steps=[('a',
                                                                   SimpleImputer(add_indicator=False,
                                                                                 copy=True,
                                                                                 fill_value=None,
                                                                                 missing_values=nan,
                                                                                 strategy='median',
                                                                   

In [127]:
preds_train = my_pipeline.predict(train_x)
error_train = mean_absolute_error(preds_train, train_y)
print("Error en el dataset de train:", error_train)
preds_test = my_pipeline.predict(test_x)
error_test = mean_absolute_error(preds_test, test_y)
print("Error en el dataset de test:", error_test)
print("Diferencia: ", error_test - error_train)

Error en el dataset de train: 505239.53528389486
Error en el dataset de test: 529861.2055695597
Diferencia:  24621.670285664848


In [121]:
from sklearn.ensemble import RandomForestClassifier
RF_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('classifier', RandomForestClassifier(n_estimators = 1000, max_depth = 5))
                             ])

In [ ]:
#X_train,X_test,y_train,y_test = train_test_split(train_x, train_y,test_size=0.3, random_state = 0)
#clf = RF_pipeline.fit(X_train, y_train)
#importancia_de_features = RF_pipeline.steps[1][1].feature_importances_
#for feature in sorted(zip(importancia_de_features,features),reverse=True):
  #print(feature)

In [ ]:
train_x.tipodepropiedad = train_x.tipodepropiedad.astype(str)
train_x.provincia = train_x.provincia.astype(str)
my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', model)
                             ])

In [ ]:
my_pipeline.fit(train_x, train_y)

In [ ]:
preds = my_pipeline.predict(train_x)

In [ ]:
ping()
mean_absolute_error(preds, train_y)

In [ ]:
preds_finales = my_pipeline.predict(prueba)

In [ ]:
res = pd.DataFrame(preds_finales, index=prueba.id, columns=['precio'])
res.reset_index(inplace=True)
res.columns = ["id", "target"]
display(res.head())
# RMSLE=1.0249284784393988 ?


# import the modules we'll need
from IPython.display import HTML
import base64

# function that takes in a dataframe and creates a text link to  
# download it (will only work for files < 2MB or so)
def create_download_link(df, title = "Download CSV file", filename = "knn-results.csv"):  
    csv = df.to_csv(index=False)
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)

# create a link to download the dataframe
create_download_link(res)

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(train_x, train_y,train_size=0.8, random_state = 0)
x_train_procesado = preprocessor.fit_transform(x_train)
x_val_procesado = preprocessor.fit_transform(x_val)
my_pipeline.fit(x_train,y_train, model__eval_set=[(x_train_procesado, y_train.to_numpy()), (x_val_procesado, y_val.to_numpy())])
preds = my_pipeline.predict(x_val)
predictions = [round(value) for value in preds]



In [ ]:
results = model.evals_result()
display("Maxima diferencia al final: " + str(results['validation_1']['mae'][-1] - results['validation_0']['mae'][-1]))
epochs = len(results['validation_0']['mae'])
x_axis = range(0, epochs)
# plot log loss
fig, ax = plt.subplots()
ax.plot(x_axis, results['validation_0']['mae'], label='Train')
ax.plot(x_axis, results['validation_1']['mae'], label='Test')
ax.legend()
plt.ylabel('mae')
plt.title('XGBoost mae')

In [ ]:
from scipy import stats
from sklearn.model_selection import KFold
from sklearn.model_selection import RandomizedSearchCV
param_dist = {'n_estimators': stats.randint(600, 2000),
              'learning_rate': stats.uniform(0.01, 0.1),
              'subsample': stats.uniform(0.3, 0.7),
              'max_depth': [3, 4, 5, 6, 7],
              'colsample_bytree': stats.uniform(0.5, 0.45),
              'min_child_weight': [3, 4, 5, 6]
             }
model = xgb.XGBRegressor(tree_method='gpu_hist', gpu_id=0, eval_metric = "mae")
clf = RandomizedSearchCV(model, param_distributions = param_dist, n_iter = 5, error_score = 0, verbose = 3)

numFolds = 5
folds = KFold(n_splits = numFolds, shuffle = True)

entrenamiento = df.head(240000).copy()
X = entrenamiento[features].copy()
y = entrenamiento.precio.copy()
estimators = []
results = np.zeros(len(train_x))
score = 0.0
for train_index, test_index in folds.split(X):
    X_train, X_test = X.iloc[train_index,:], X.iloc[test_index,:]
    y_train, y_test = y.iloc[train_index].values.ravel(), y.iloc[test_index].values.ravel()
    x_train_procesado = preprocessor.fit_transform(X_train)
    x_val_procesado = preprocessor.fit_transform(X_test)
#    my_pipeline.fit(X_train, y_train)
    clf.fit(x_train_procesado, y_train)

    estimators.append(clf.best_estimator_)
#    estimators.append(my_pipeline)
#    results[test_index] = my_pipeline.predict(X_test)
    results[test_index] = clf.predict(x_val_procesado)
    score += mean_absolute_error(y_test, results[test_index])
score /= numFolds
ping()
score

In [ ]:
mean_absolute_error(results, y)

In [ ]:
estimators